In [ ]:
import psycopg2

try:
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Versão do PostgreSQL:", version)

except Exception as error:
    print("Erro ao conectar ao banco de dados:", str(error).encode('utf-8', errors='replace').decode())


In [ ]:
import requests
import psycopg2  

connection =  connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8" 
    )
cursor = connection.cursor()

response = requests.get('https://restcountries.com/v3.1/all?fields=name,area')
data = response.json()

for country in data:
    country_name = country['name']['common']
    area = country['area']
    
    sqlCountry = """
    INSERT INTO PUBLIC.COUNTRY (COUNTRY_NAME, ID)
    VALUES (%s, %s)
    ON CONFLICT (ID) DO NOTHING;
    """
    val = (country_name, area)  
    
    # Executando a query
    cursor.execute(sqlCountry, val)

connection.commit()
cursor.close()
connection.close()



In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2  

connection = psycopg2.connect(
    host="26.84.71.27",       
    database="DataScienceProj",
    user="postgres",     
    password="SteamDB",    
    port="5432",
    options="-c client_encoding=UTF8" 
)
cursor = connection.cursor()

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    links = soup.select('table.wikitable a[href^="/wiki/"]')  

    nomes_paises = []

    for link in links:
        nome = link.get_text(strip=True) 
        if nome:  
            nomes_paises.append(nome)

    nomes_paises_limitados = nomes_paises[:10]

    for i in range(1, len(nomes_paises_limitados) + 1):
        nome = nomes_paises_limitados[i - 1] 
        print(f"{i}: {nome}")
        cursor.execute("SELECT id FROM public.country WHERE upper(country_name) = upper(%s)", (nome,))
        resultado = cursor.fetchone()

        if resultado:
            pais_id = resultado[0]
            # Inserindo o registro
            cursor.execute("INSERT INTO public.Best_country_live (id, country_id) VALUES (%s,%s)", (i,pais_id,))
        else:
            print(f'País {nome} não encontrado no banco de dados.')

    connection.commit()
else:
    print('Erro ao acessar o site:', response.status_code)

cursor.close()
connection.close()


In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2  


connection = psycopg2.connect(
    host="26.84.71.27",       
    database="DataScienceProj",
    user="postgres",     
    password="SteamDB",    
    port="5432",
    options="-c client_encoding=UTF8" 
)
cursor = connection.cursor()


url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'


response = requests.get(url)


if response.status_code == 200:
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    
    links = soup.select('table.wikitable a[href^="/wiki/"]')  

    
    nomes_paises = []

    for link in links:
        nome = link.get_text(strip=True)  
        if nome: 
            nomes_paises.append(nome)

    nomes_paises_limitados = nomes_paises[:10]

    for i in range(1, len(nomes_paises_limitados) + 1):
        nome = nomes_paises_limitados[i-1] 
        print(f"{i}: {nome}") 
        cursor.execute("SELECT id FROM public.country WHERE upper(country_name) = upper(%s)", (nome,))
        resultado = cursor.fetchone()

        if resultado:
            pais_id = resultado[0]
            # Inserindo o registro
            cursor.execute("INSERT INTO public.Populous_countries (id, country_id) VALUES (%s,%s) on conflict (id) do nothing", (i,pais_id,))
        else:
            print(f'País {nome} não encontrado no banco de dados.')

    connection.commit()
else:
    print('Erro ao acessar o site:', response.status_code)

cursor.close()
connection.close()
